In [1]:
print("hello world!")

hello world!


In [ ]:
import tensorflow as tf
import numpy as np
import os

pwd = os.getcwd()


onehot编码可以用 np.eye 来快速生成

In [ ]:
def onehot(labels):
  labels = labels.astype(int)
  onehot_labels = np.eye(labels.shape[0], 31)[labels]
  return onehot_labels

如果序列长度不同 可以用keras 的pad_sequences 来填充

In [ ]:
def read_data(x_src, y_src):
  x = np.load(x_src)
  # x = tf.keras.preprocessing.sequence.pad_sequences(x, maxlen=20, dtype='float32',
  #                                                   padding='post', truncating='post', value=0.)
  #   nsamples, nx = x.shape
  # x = x.reshape((nsamples, nx))

  y = np.load(y_src)
  y = onehot(y)

  return x, y

In [ ]:
# 计算准确率
def compute_accurate_rate(x_dev, y_dev):
  num = x_dev.shape[0]
  total_batch = int(num / batch_size)
  score = 0
  for i in range(total_batch):
    correct_prediction = tf.equal(tf.argmax(out_layer, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    score2 = accuracy.eval(
      {x: x_dev[i * batch_size:batch_size * (i + 1)], y: y_dev[i * batch_size:batch_size * (i + 1)]})
    score += score2

  return score / total_batch


In [ ]:
input_shape = 2675
learning_rate = 0.0001
training_epochs = 5000
batch_size = 75
# 要调的layer大小
hidden_layer_sizes = [[512, 256], [256, 128], [256, 64], [128, 64]]


 tf.reset_default_graph() 是为了清除前面的计算图，会把之前的变量这些都清了<br>
 hparam_str 是为了拼接字符串，然后输出的tensorboard的时候好看，能对比区别<br>
 Variable 和get_variable(不记得是不是这样拼了，没代码补充提示)，前面的是直接创建，后面的是回去搜索一遍不重复(前提:name_scope的reuse=True的话,不是就直接创建)<br>
 tf.summary.scalar 添加向量到 summary中<br>
 merged_summary = tf.summary.merge_all() 把添加的向量全部集合起来<br>
 然后sess.run的时候run吧merged_summary也run一下。<br>
 并且把输出ms到writer.add_summary(ms, epoch)<br>
 writer 是 writer = tf.summary.FileWriter("/home/bringtree/data/mlp_demo/" + hparam_str)<br>
 添加操作图writer.add_graph(sess.graph)<br>
 
 会遇到的问题:<br>
 1. 如下面的情况 有时候 会用留1法等等来训练超参，这时候writer会出现覆盖的现象，记得改文件名,清graph.<br>
 2. 不同色文件名 颜色不同 不能自定义<br>

In [ ]:
for cell in hidden_layer_sizes:
  tf.reset_default_graph()
  hparam_str = "cell_" + str(cell[0]) + '_' + str(cell[1])

  x = tf.placeholder('float32', [batch_size, input_shape], name="x")
  y = tf.placeholder('float32', [batch_size, 31], name="labels")
  with tf.name_scope("input_layer"):
    w_1 = tf.Variable(tf.random_uniform([input_shape, cell[0]], dtype='float32'), name="w_1")
    b_1 = tf.Variable(tf.random_uniform([cell[0]], dtype='float32'), name="b_1")
    layer1 = tf.matmul(x, w_1)
    layer1 = tf.add(layer1, b_1)
    layer1 = tf.contrib.layers.batch_norm(layer1, decay=0.9, updates_collections=None, epsilon=1e-5, scale=True)
    layer1 = tf.nn.relu(layer1)

  with tf.name_scope("mid_layer"):
    w_2 = tf.Variable(tf.random_uniform([cell[0], cell[1]], dtype='float32'), name="w_2")
    b_2 = tf.Variable(tf.random_uniform([cell[1]], dtype='float32'), name="b_2")
    layer2 = tf.matmul(layer1, w_2)
    layer2 = tf.add(layer2, b_2)
    layer2 = tf.contrib.layers.batch_norm(layer2, decay=0.9, updates_collections=None, epsilon=1e-5, scale=True)
    layer3 = tf.nn.relu(layer2)

  with tf.name_scope("output_layer"):
    w_3 = tf.Variable(tf.random_uniform([cell[1], 31], dtype='float32'), name="w_3")
    b_3 = tf.Variable(tf.random_uniform([31], dtype='float32'), name="b_3")
    out_layer = tf.add(tf.matmul(layer3, w_3), b_3)

  with tf.name_scope("lost_function"):
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=out_layer))

  with tf.name_scope("train_optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

  # 训练模型
  model_accuracy = 0
  for data_index in range(1, 5):
    with tf.name_scope("accuracy"):
      correct_prediction = tf.equal(tf.argmax(out_layer, 1), tf.argmax(y, 1))
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
      tf.summary.scalar('accuracy' + str(data_index), accuracy)

    with tf.Session() as sess:
      writer = tf.summary.FileWriter("/home/bringtree/data/mlp_demo/" + hparam_str)
      writer.add_graph(sess.graph)
      init = tf.global_variables_initializer()
      sess.run(init)
      merged_summary = tf.summary.merge_all()

      for epoch in range(1, training_epochs + 1):
        x_train, y_train = read_data(
          pwd + "/SMP2017/smp2017_5fold/" + str(data_index) + "_train_test_x_train_kf_smp_bow.npy",
          pwd + "/SMP2017/smp2017_5fold/" + str(data_index) + "_train_test_y_train_kf_smp_bow.npy")

        avg_cost = 0
        n_sample = x_train.shape[0]
        total_batch = int(n_sample / batch_size)

        for i in range(total_batch):
          _, c = sess.run([optimizer, cost], feed_dict={x: x_train[i * batch_size: (i + 1) * batch_size],
                                                        y: y_train[i * batch_size: (i + 1) * batch_size]})
          avg_cost += c / total_batch

        if epoch % 5 == 0:
          # accurate_rate
          x_dev, y_dev = read_data(
            pwd + "/SMP2017/smp2017_5fold/" + str(data_index) + "_train_test_x_test_kf_smp_bow.npy",
            pwd + "/SMP2017/smp2017_5fold/" + str(data_index) + "_train_test_y_test_kf_smp_bow.npy")

          num = x_dev.shape[0]
          total_batch = int(num / batch_size)
          score = 0
          for i in range(total_batch):
            score2, ms = sess.run([accuracy, merged_summary],
                                  feed_dict={x: x_dev[i * batch_size:batch_size * (i + 1)],
                                             y: y_dev[i * batch_size:batch_size * (i + 1)]})
            score += score2
          writer.add_summary(ms, epoch)

          accuracy_average = score / total_batch

          # print('Epoch:', '%04d' % (epoch), "Accuracy:" + str(accuracy_average))
          # print('Epoch:', '%04d' % (epoch), 'cost=', '{:.9f}'.format(avg_cost))

      x_dev, y_dev = read_data(
        pwd + "/SMP2017/smp2017_5fold/" + str(data_index) + "_train_test_x_test_kf_smp_bow.npy",
        pwd + "/SMP2017/smp2017_5fold/" + str(data_index) + "_train_test_y_test_kf_smp_bow.npy")
      model_accuracy += compute_accurate_rate(x_dev, y_dev)

  print(str(model_accuracy / 4) + ',' + str(cell) + ',' + "\n")
